<a href="https://colab.research.google.com/github/KacperGrodecki/nieruchomosci-mazowieckie/blob/0.0.4/mazowieckie_colab_rysunki_i_analiza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [380]:
from sklearn.metrics import confusion_matrix
from sklearn import svm
import pandas as pd
import numpy as np
np.random.seed(7)
from google.colab import drive
import seaborn as sns
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import pandas as pd
#from otoDomScraper import daneDomu
#from random import randrange
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import statistics
from sklearn.cluster import AffinityPropagation
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.decomposition import FastICA
from sklearn import preprocessing
from IPython.display import Javascript
import requests
from collections import OrderedDict
import seaborn as sns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [381]:
def toNum2(txt):
    if type(txt) is int:
        return txt
    elif (type(txt) is str):
        digs = re.findall(r'\d+', txt)
        if len(digs) == 1:
            return int(digs[0])
        elif len(digs) == 2:
            return 1000 * int(digs[0]) + int(digs[1])
        elif len(digs) == 3:
            return 1000000 * int(digs[0]) + 1000 * int(digs[1]) + int(digs[0])

    #   return int(digs)


def toNum1(txt):
    if type(txt) is str:
        digs = re.findall(r'\d+', txt)
        if len(digs) == 1:
            return int(digs[0])
        elif len(digs) == 2 and (txt[1] != ' '):
            return int(digs[0]) + 0.01 * int(digs[1])
        elif len(digs) == 3:
            return 1000 * int(digs[0]) + int(digs[1]) + 0.001 * int(digs[2])
        elif (type(txt) is str) and (txt[1] == ' '):
            digs = re.findall(r'\d+', txt)
            return 1000 * int(digs[0]) + int(digs[1])
    else:
        return txt


def toNum3(txt):
    if type(txt) == int:
        return txt
    return int(re.findall(r'\d+', txt)[0])

def pietra(txt):
    if type(txt) is str:
        if '0' in txt:
            return 0
        if '1' in txt:
            return 1
        elif '2' in txt:
            return 2
        elif '3' in txt:
            return 3
        elif 'parterowy' in txt:
            return 0
    else:
        return txt

def cities(x):
    dist=x.split()[4]
    #city=x.split()[5]
    if dist=='warszawski':
        return x.split()[6]
    elif dist in ['Warszawa','Radom','Płock','Siedlce']:
        return dist
    else:
        try:
            return x.split()[5]
        except:
            return 'unknown'

def region(x):
    if x.split()[4]=='Warszawa':
        try:
            return x.split()[5]
        except:
            return ''
    else:
        return ''

In [382]:
#code written by Maciej Kozubaldef get_voivodeships():
    # problem - too many voivodeships, historical ones are mixed together
    # solution - just cut the list after 16-th item
    
    # request voivodeships of poland(wd:Q36),
    # cut after 16 voivodeships, the rest are historical ones
    # https://query.wikidata.org/#%20%20%20%20SELECT%20%3Fvoivodeship%20%3FvoivodeshipLabel%20%3Flatitude%20%3Flongitude%20%3Fadmininistrative_teritorial_entity%0A%20%20%20%20WHERE%20%7B%0A%20%20%20%20%20%20%3Fvoivodeship%20wdt%3AP31%20wd%3AQ150093%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP131%20%3Fadmininistrative_teritorial_entity%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20p%3AP625%2Fpsv%3AP625%20%5B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLatitude%20%3Flatitude%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLongitude%20%3Flongitude%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%5D.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20FILTER%28%3Fadmininistrative_teritorial_entity%20%3D%20wd%3AQ36%29.%0A%20%20%20%20%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22pl%22.%20%7D%0A%20%20%20%20%7D%0A%20%20%20%20ORDER%20BY%20DESC%28%3Fvoivodeship%29%0A%20%20%20%20LIMIT%2016%0A
    url = 'https://query.wikidata.org/sparql'
    query = '''
    SELECT ?voivodeship ?voivodeshipLabel ?latitude ?longitude ?admininistrative_teritorial_entity
    WHERE {
      ?voivodeship wdt:P31 wd:Q150093;
                   wdt:P131 ?admininistrative_teritorial_entity;
                   p:P625/psv:P625 [
                       wikibase:geoLatitude ?latitude ;
                       wikibase:geoLongitude ?longitude ;
                   ].               
      FILTER(?admininistrative_teritorial_entity = wd:Q36).
      SERVICE wikibase:label { bd:serviceParam wikibase:language "pl". }
    }
    ORDER BY DESC(?voivodeship)
    LIMIT 16
    '''
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()

    # convert json to dataframe
    voivodeships = []
    for item in data['results']['bindings']:
        voivodeships.append(OrderedDict(
        {
            'voivodeship':      item['voivodeshipLabel']['value'].lower(),
            'latitude':         float(item['latitude']['value']),        
            'longitude':        float(item['longitude']['value']),                    
            'wikidata_item_id': item['voivodeship']['value'].split('/')[-1]
        }))
    #     print(item,'\n')    
    return pd.DataFrame(voivodeships)


In [383]:
#code written by Maciej Kozubaldef get_warsaw_districts():
    # problem  - some districts have two sets of coordinates which differ slighlty
    # solution - drop the one with worse precision (larger value), this seems to be consistent with the wikipedia data
    # to do    - the above is not true for ['Wola'], change it
    
    # https://query.wikidata.org/#SELECT%20%3Fdistrict_of_Warsaw%20%3Fdistrict_of_WarsawLabel%20%3Flat%20%3Flon%20%3FgeoPrecision%20%0AWHERE%20%7B%0A%20%20%3Fdistrict_of_Warsaw%20%20wdt%3AP31%20wd%3AQ4286337%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP17%20wd%3AQ36%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP131%20wd%3AQ270%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20p%3AP625%2Fpsv%3AP625%20%5B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLatitude%20%3Flat%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLongitude%20%3Flon%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoPrecision%20%20%3FgeoPrecision%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%5D%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22pl%22.%20%7D%0A%7D
    url = 'https://query.wikidata.org/sparql'
    query = '''
    SELECT ?warsaw_district ?warsaw_districtLabel ?latitude ?longitude ?geoPrecision 
    WHERE {
      ?warsaw_district  wdt:P31 wd:Q4286337;
                           wdt:P17 wd:Q36;
                           wdt:P131 wd:Q270;
                           p:P625/psv:P625 [
                               wikibase:geoLatitude ?latitude ;
                               wikibase:geoLongitude ?longitude ;
                               wikibase:geoPrecision  ?geoPrecision;                                       
                           ]
      SERVICE wikibase:label { bd:serviceParam wikibase:language "pl". }
    }
    ORDER BY ASC(?warsaw_districtLabel)
    '''
    r = requests.get(url, params = {'format': 'json', 'query': query})
#     print(r.json())
    data = r.json()

    # convert json to dataframe
    warsaw_districts = []
    for item in data['results']['bindings']:
#         print(item,'\n')    
        warsaw_districts.append(OrderedDict(
        {
            'warsaw_district':  item['warsaw_districtLabel']['value'].lower(),
            'latitude':         float(item['latitude']['value']),        
            'longitude':        float(item['longitude']['value']),                    
            'geoPrecision':     float(item['geoPrecision']['value']),                                
            'wikidata_item_id': item['warsaw_district']['value'].split('/')[-1]
        }))
    
    warsaw_districts = pd.DataFrame(warsaw_districts).sort_values(by=['warsaw_district'])
    
    # if duplicate warsaw districts exist, take the one with better precision (lower value), do the opposite in case of 'Wola' 
    for district in warsaw_districts['warsaw_district']:
        if np.sum(warsaw_districts['warsaw_district'] == district) > 1: # duplicate district found
            if district != 'wola':            
                district_to_drop_idx = warsaw_districts.loc[warsaw_districts['warsaw_district'] == district, 'geoPrecision'].idxmax()
            elif district == 'wola':
                district_to_drop_idx = warsaw_districts.loc[warsaw_districts['warsaw_district'] == district, 'geoPrecision'].idxmin()
#             print(district+':\t', district_to_drop_idx, '\n')
            warsaw_districts = warsaw_districts.drop(district_to_drop_idx).reset_index(drop=True)  
            
    return warsaw_districts

In [384]:
#code written by Maciej Kozubaldef get_cities():
    # Description:
    #  - sparql query is constructed for different types of cities: city with powiat rights ('Q925381'), urban municipality of Poland (Q2616791), and the second for "regular" city ('Q515')
    # Problems:
    #   1) some cities have more than one set of coordinates
    #   2) filter within masovian voivodeship
    # Solutions:
    #   1) retain only the one with best precision (lowest value)
    #   2) we can use .contains method from geopandas package, check the crs of Point from wikidata (https://www.kaggle.com/alexisbcook/proximity-analysis)
    # To do:
    #   1) check this approach
      
    # Get both types of cities: cities with powiat rights (Q925381), urban municipality of Poland (Q2616791), and the the "regular" cities (Q515)
    # https://docs.python.org/3/reference/lexical_analysis.html#f-strings
    # https://query.wikidata.org/#SELECT%20%3Fcity%20%3FcityLabel%20%3Flatitude%20%3Flongitude%20%3FgeoPrecision%20%3Fcoord%0AWHERE%20%7B%0A%20%20%3Fcity%20%20wdt%3AP31%20wd%3AQ925381%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP17%20wd%3AQ36%3B%0A%20%20%20%20%20%20%20%20%20%23wdt%3AP131%20wd%3AQ54169%3B%0A%20%20%20%20%20%20%20%20%20p%3AP625%20%5B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20psv%3AP625%20%5B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLatitude%20%3Flatitude%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLongitude%20%3Flongitude%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoPrecision%20%20%3FgeoPrecision%3B%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%5D%3B%0A%20%20%20%20%20%20%20%20%20ps%3AP625%20%3Fcoord%20%0A%20%20%20%20%20%20%20%20%20%5D%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22pl%22.%20%7D%0A%7D
    # https://query.wikidata.org/#SELECT%20%3Fcity%20%3FcityLabel%20%3Flatitude%20%3Flongitude%20%3FgeoPrecision%20%3Fcoord%0AWHERE%20%7B%0A%20%20%3Fcity%20%20wdt%3AP31%20wd%3AQ515%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP17%20wd%3AQ36%3B%0A%20%20%20%20%20%20%20%20%20%23wdt%3AP131%20wd%3AQ54169%3B%0A%20%20%20%20%20%20%20%20%20p%3AP625%20%5B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20psv%3AP625%20%5B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLatitude%20%3Flatitude%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLongitude%20%3Flongitude%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoPrecision%20%20%3FgeoPrecision%3B%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%5D%3B%0A%20%20%20%20%20%20%20%20%20ps%3AP625%20%3Fcoord%20%0A%20%20%20%20%20%20%20%20%20%5D%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22pl%22.%20%7D%0A%7D
    cities_all = pd.DataFrame()
    url = 'https://query.wikidata.org/sparql'
    data = pd.DataFrame()
    types_of_city = {'city with powiat rights': 'Q925381', 'urban municipality of Poland ': 'Q2616791', 'city': 'Q515'}
    for type_of_city, type_of_city_sparql in types_of_city.items():
#         print('\n\nCities of type \'' + type_of_city + '\' (' + str(type_of_city_sparql) + '):')
        query = f''' 
            SELECT ?city ?cityLabel ?latitude ?longitude ?geoPrecision ?coord
            WHERE {{
              ?city  wdt:P31 wd:{type_of_city_sparql};
                     wdt:P17 wd:Q36;
                     #wdt:P131 wd:Q54169;
                     p:P625 [
                             psv:P625 [
                                       wikibase:geoLatitude ?latitude ;
                                       wikibase:geoLongitude ?longitude ;
                                       wikibase:geoPrecision  ?geoPrecision;      
                                      ];
                     ps:P625 ?coord 
                     ]
              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "pl". }}
            }}
            '''
        r = requests.get(url, params = {'format': 'json', 'query': query})
        data = r.json()

        
        # Convert json to dataframe
        cities = []
        for item in data['results']['bindings']:
#             print(item,'\n')    
            cities.append(OrderedDict(
            {
                'city':             item['cityLabel']['value'].lower(),
                'latitude':         float(item['latitude']['value']),        
                'longitude':        float(item['longitude']['value']),                    
                'geoPrecision':     float(item['geoPrecision']['value']),                                
                'wikidata_item_id': item['city']['value'].split('/')[-1]
            }))
        cities = pd.DataFrame(cities).sort_values(by=['city']).reset_index(drop=True)
#         print('Number of cities of type', type_of_city+':', len(cities))

        
        # Find those city names that have multiple instaces:
        #   - those with the same wikidata_item_id are the same city with multiple coordinate sets - retain only the set with best (lowest) geoPrecision,
        #   - those with different wikidata_item_id are different cities and should not be considered with the following procedure.
        # https://stackoverflow.com/questions/55360314/pandas-groupby-take-counts-greater-than-1
        cities_with_the_same_name_and_wikidataItemId = cities.loc[cities.groupby(['city', 'wikidata_item_id'])['geoPrecision'].transform('count') > 1].reset_index(drop=True) # cities.loc[cities.duplicated(subset=['city', 'wikidata_item_id'], keep=False)].reset_index(drop=True)
#         print('Cities with the same name and the same wikidata_item_id:', len(cities_with_the_same_name_and_wikidataItemId))
        
    
        # If multiple instances of any city exist, retain only the one with the best precision (lowest value) 
#         print('Cities with multiple instances:', len(cities_with_the_same_name_and_wikidataItemId), '\n')
        for city in cities_with_the_same_name_and_wikidataItemId['city'].unique():
#             print(city)
            city_to_retain_idx = cities_with_the_same_name_and_wikidataItemId.loc[cities_with_the_same_name_and_wikidataItemId['city'] == city, 'geoPrecision'].idxmin()        
            cities_to_drop = cities_with_the_same_name_and_wikidataItemId.loc[(cities_with_the_same_name_and_wikidataItemId['city'] == city) & (cities_with_the_same_name_and_wikidataItemId.index != city_to_retain_idx)].index
#             print('city:', city, '\nindex and geoPrecision of instance to retain:', city_to_retain_idx, cities_with_the_same_name_and_wikidataItemId.loc[city_to_retain_idx, 'geoPrecision'], '\nindices and geoPrecisions of instances to drop:\n', tabulate(cities_with_the_same_name_and_wikidataItemId.loc[cities_to_drop, ['geoPrecision']], tablefmt='psql'))
            cities = cities.drop(cities_to_drop).reset_index(drop=True)
#         print('Number of cities with multiple instances of given cities after cleaning:', len(cities_with_the_same_name_and_wikidataItemId))


        # Append to cities_all
        cities_all = pd.concat([cities_all, cities], axis = 0).sort_values(by=['city']).reset_index(drop=True)
#         print('Number of cities of type', type_of_city, 'after cleaning duplicate coordinates:', len(cities))

    
#     print('Number of all cities:', len(cities_all), '\n', '#'*72)
    return cities_all

In [385]:
#code written by Maciej Kozubaldef get_counties():
    # Description:
    #  - sparql query is constructed for counties ('Q247073'),
      
    # Get both types of cities: cities with powiat rights (Q925381), urban municipality of Poland (Q2616791), and the the "regular" cities (Q515)
    # https://docs.python.org/3/reference/lexical_analysis.html#f-strings
    # https://query.wikidata.org/#SELECT%20%3Fcity%20%3FcityLabel%20%3Flatitude%20%3Flongitude%20%3FgeoPrecision%20%3Fcoord%0AWHERE%20%7B%0A%20%20%3Fcity%20%20wdt%3AP31%20wd%3AQ247073%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP17%20wd%3AQ36%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP131%20wd%3AQ54169%3B%0A%20%20%20%20%20%20%20%20%20p%3AP625%20%5B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20psv%3AP625%20%5B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLatitude%20%3Flatitude%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLongitude%20%3Flongitude%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoPrecision%20%20%3FgeoPrecision%3B%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%5D%3B%0A%20%20%20%20%20%20%20%20%20ps%3AP625%20%3Fcoord%20%0A%20%20%20%20%20%20%20%20%20%5D%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22pl%22.%20%7D%0A%7D
    cities_all = pd.DataFrame()
    url = 'https://query.wikidata.org/sparql'
    query = '''
    SELECT ?county ?countyLabel ?latitude ?longitude ?geoPrecision ?coord
    WHERE {
      ?county  wdt:P31 wd:Q247073;
            wdt:P17 wd:Q36;
            wdt:P131 wd:Q54169;
            p:P625 [
                    psv:P625 [
                              wikibase:geoLatitude ?latitude ;
                              wikibase:geoLongitude ?longitude ;
                              wikibase:geoPrecision  ?geoPrecision;      
                              ];
            ps:P625 ?coord 
            ]
      SERVICE wikibase:label { bd:serviceParam wikibase:language "pl". }
    }
    '''
    r = requests.get(url, params = {'format': 'json', 'query': query})
#     print(r.json())
    data = r.json()

    # convert json to dataframe
    counties = []
    for item in data['results']['bindings']:
#         print(item,'\n')    
        counties.append(OrderedDict(
        {
            'county':           item['countyLabel']['value'].lower(),
            'latitude':         float(item['latitude']['value']),        
            'longitude':        float(item['longitude']['value']),                    
            'geoPrecision':     float(item['geoPrecision']['value']),                                
            'wikidata_item_id': item['county']['value'].split('/')[-1]
        }))
    
    counties = pd.DataFrame(counties).sort_values(by=['county'])
      
    return counties

In [386]:
#dfCities=get_cities()
#dfCounties=get_counties()
#dfVoivodeships=get_voivodeships()
#dfWarsawDistricts=get_warsaw_districts()

In [387]:
#code written by Maciej Kozubaldef return_coordinates(place,type):
    '''
    Returns coordinates of the place.
        The place must be the name (in polish) of voivodeship in Poland, city, or the district of Warsaw.
    '''
    
    coordinates = [None, None]
    place = place.lower()
    if type=='V':
      print('voivodeships',place)

      try:
          voivodeships = get_voivodeships()
          coordinates_df = voivodeships.loc[voivodeships['voivodeship'].str.contains(place) == True, ['latitude', 'longitude']]
          coordinates = coordinates_df.values.tolist()[0]       
      except Exception as e:
        print(e)
    elif type=='D':
      print('warsaw districts',place)
      try:
          counties = get_counties()
          coordinates_df = counties.loc[counties['warsaw_district'].str.contains(place) == True, ['latitude', 'longitude']]
          coordinates = coordinates_df.values.tolist()[0]
      except Exception as e:
          print(e)
    elif type=='C':   
      print('cities',place)
      try:
          cities = get_cities()
          coordinates_df = cities.loc[cities['city'].str.contains(place) == True, ['latitude', 'longitude']]
          coordinates = coordinates_df.values.tolist()[0]
      except Exception as e:
          print(e)   
    print(coordinates)
    return coordinates

In [388]:
def makeDataFrame(file):
    dfMazowieckie=pd.read_csv(file,index_col=0)
    concat=dfMazowieckie
    concat['powierzchnia_corr'] = concat['powierzchnia'].apply(lambda x: toNum1(x))
    concat['powierzchniaDzialki_corr'] = concat['powierzchniaDzialki'].apply(lambda x: toNum2(x))
    concat['cena_corr'] = concat['cena'].apply(lambda x: toNum2(x))
    concat['rokBudowy_corr'] = concat['rokBudowy'].apply(lambda x: toNum3(x))
    concat['cena/m'] = concat['cena_corr'] / concat['powierzchnia_corr']
    concat['lPieter_crr'] = concat['lPieter'].apply(lambda x: pietra(x))
    concat = concat[concat['powierzchnia_corr'] > 0]
    concat = concat[concat['cena_corr'] > 0]
    concat['lPokoi'] = concat['lPokoi'].apply(lambda x: toNum3(x))
    concat['districts']=concat['dzielnica'].apply(lambda x: x.split()[4])
    cities_corr=concat['dzielnica'].apply(lambda x: cities(x))
    concat['cities_corr']=cities_corr
    region_corr=concat['dzielnica'].apply(lambda x: region(x))
    concat['region_corr']=region_corr
    concat_dropped = concat.drop(['dzielnica', 'powierzchnia', 'powierzchniaDzialki', 'lPieter', 'cena', 'cena_corr',], axis=1)
    #['rokBudowy_corr'] = concat_dropped['rokBudowy_corr'].replace(to_replace=0, value=1990)
    concat_dropped['rokBudowy_corr'] = concat_dropped['rokBudowy_corr'].astype('int')
    concat_dropped.loc[concat_dropped['rokBudowy_corr']<1900,'rokBudowy_corr']=1980
    concat_dropped.loc[concat_dropped['rokBudowy_corr']>2030,'rokBudowy_corr']=2020
    #concat_dropped = concat_dropped[concat_dropped['rokBudowy_corr'] < 2030]
    concat_dropped = concat_dropped.fillna(0)
    concat_dropped = concat_dropped[concat_dropped['cena/m'] < 20000]
    concat_dropped['cena/m'].hist(bins=200)

    #concat_dropped_dumm = pd.get_dummies(concat_dropped, columns=['rodzajZabudowy', 'materialBudynku', 'stanWykonczenia', 'okna', 'rynek',
    #                                       'cities_corr','districts','region_corr'])
    
    cityData=concat_dropped.cities_corr.apply(lambda x: funCities(x))
    locationCities=locCities(cityData)
    districtsData=concat_dropped.region_corr.apply(lambda x: funDistricts(x))
    locationDistricts=locCities(districtsData)
    countyData=concat_dropped.districts.apply(lambda x: funCounties(x))
    locationCounty=locCounties(countyData)

    concat_dropped_reset=concat_dropped.reset_index().drop(['index'],axis=1)
    locations= pd.concat([locationCities,locationDistricts,locationCounty], axis=1)
    concat_dropped_reset['locationX']=locations.apply(selectLocationX,axis=1)
    concat_dropped_reset['locationY']=locations.apply(selectLocationY,axis=1)
    concat_dropped_reset_drop = concat_dropped_reset.drop(['rokBudowy', 'districts', 'cities_corr', 'region_corr'], axis=1)
    

    return concat_dropped_reset_drop

In [389]:
def funCities(x):
  rows=dfCities.loc[dfCities['city']==x.lower()]
  precision=0
  if rows.shape[0]==0:
    return 0,0,100
  elif rows.shape[0]==1:
    longitude=rows.longitude.values
    latitude=rows.latitude.values
    precision=rows.geoPrecision.values
    return longitude[0],latitude[0],precision[0]
  elif rows.shape[0]>1:
    prec1,prec2=rows.iloc[0,3],rows.iloc[1,3]
    if prec1<prec2:
      precision=prec1
      longitude=rows.iloc[0,2]
      latitude=rows.iloc[0,1]
    else:
      precision=prec2
      longitude=rows.iloc[1,2]
      latitude=rows.iloc[1,1]
    return longitude,latitude,precision

In [390]:
def funDistricts(x):
  rows=dfWarsawDistricts.loc[dfWarsawDistricts['warsaw_district']==x.lower()]
  #print(x, ' ',rows.shape[0])
  if rows.shape[0]==0:
    return 0,0,100
  elif rows.shape[0]==1:
    longitude=rows.longitude.values
    latitude=rows.latitude.values
    precision=rows.geoPrecision.values
    return longitude[0],latitude[0],precision[0]

In [391]:
def funCounties(x):
  try:
    rows=dfCounties.loc['powiat '+x.lower()== dfCounties['county']]
  except Exception as e:
    return e
  #print(x, ' ',rows.shape[0])
  if rows.shape[0]==0:
    return 0,0,100
  elif rows.shape[0]==1:
    longitude=rows.longitude.values
    latitude=rows.latitude.values
    precision=rows.geoPrecision.values
    return longitude[0],latitude[0],precision[0]

In [392]:
def selectLocationX(x):
 # print(x[17],x[20],x[23])
  if x[2]<min(x[5],x[8]):
    #print('min 17',x[15],x[16])
    return x[0]
  elif x[5]<min(x[2],x[8]):
    #print('min 20 ',x[18],x[19])
    return x[3]
  elif x[8]<min(x[2],x[5]):
    #print('min 23 ',x[21],x[22])
    return x[6]

def selectLocationY(x):
 # print(x[17],x[20],x[23])
  if x[2]<min(x[5],x[8]):
    #print('min 17',x[15],x[16])
    return x[1]
  elif x[5]<min(x[2],x[8]):
    #print('min 20 ',x[18],x[19])
    return x[4]
  elif x[8]<min(x[2],x[5]):
    #print('min 23 ',x[21],x[22])
    return x[7]

In [393]:
def locCities(cityData):
  locationCities=[]
  for data in cityData:
    locationCities.append(data)
  locationCities=pd.DataFrame(np.array(locationCities),columns=['cityX','cityY','cityGeo'])
  return locationCities

def locDistricts(districtsData):
  locationCities=[]
  for data in cityData:
    locationDistricts.append(data)
  locationDistricts=pd.DataFrame(np.array(locationDistricts),columns=['cityX','cityY','cityGeo'])
  return locationDistricts

def locCounties(countyData):
  locationCounty=[]
  for data in countyData:
    locationCounty.append(data)
  locationCounty=pd.DataFrame(np.array(locationCounty),columns=['countyX','countyY','countyGeo'])
  return locationCounty

In [396]:
filename='/content/drive/My Drive/domymazowieckie/dfMazowieckie.csv'
dfCities=get_cities()
dfCounties=get_counties()
dfVoivodeships=get_voivodeships()
dfWarsawDistricts=get_warsaw_districts()

ChunkedEncodingError: ignored

In [ ]:
final=makeDataFrame(filename)

In [ ]:
filename_json='/content/drive/My Drive/domymazowieckie/dfMazowieckie.json'

In [ ]:
final.to_json(filename_json)

In [ ]:
from sklearn import preprocessing

In [ ]:
plt.hist(final.loc[:,'lPokoi']/10)

In [ ]:
sns.boxplot(x=(np.log(final["powierzchnia_corr"])/10))

In [ ]:
sns.boxplot(x=(np.log(final["powierzchniaDzialki_corr"]+1)/14))

In [ ]:
sns.boxplot(x=(np.power(final["rokBudowy_corr"]-1899,4)/3e8))

In [ ]:
plt.hist(np.power(final["rokBudowy_corr"]-1899,4)/3e8)

In [ ]:
sns.boxplot(x=final["cena/m"]/20000)

In [ ]:
sns.boxplot(x=final["lPieter_crr"]/10)

In [ ]:
plt.hist(final["lPieter_crr"]/10)

In [ ]:
plt.hist((final["locationX"]-21)/4)

In [ ]:
plt.hist((final["locationY"]-52)/2)

In [ ]:
final.head()

In [ ]:
final=final[final['powierzchnia_corr']<500]
final=final[final['powierzchniaDzialki_corr']<5000]

In [ ]:
final['powierzchnia_corr'].hist()

In [ ]:
final_plots=final.copy()

In [ ]:
dummies = pd.get_dummies(final_plots, columns=['rodzajZabudowy', 'materialBudynku', 'stanWykonczenia', 'okna', 'rynek'])

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
final_plots.head()

In [ ]:
df=pd.DataFrame()

In [ ]:
final_plots1=final_plots[['powierzchnia_corr','cena/m']].copy()#'lPokoi','lPieter_crr',,'powierzchniaDzialki_corr','rokBudowy_corr',,'locationX','locationY'
#df['lPokoi']=final_plots.loc[:,'lPokoi']/10
df["powierzchnia_corr"]=np.log(final_plots1["powierzchnia_corr"])/3-1
#df["powierzchniaDzialki_corr"]=np.log(final_plots["powierzchniaDzialki_corr"]+1)/14
#df["rokBudowy_corr"]=np.power(final_plots["rokBudowy_corr"]-1899,4)/3e8
df["cena/m"]=final_plots1["cena/m"]/20000
#df["locationX"]=(final_plots["locationX"]-21)/4
#df["locationY"]=(final_plots["locationY"]-52)/4
#df['lPieter_crr']=final_plots.loc[:,'lPieter_crr']/10

In [ ]:
df["powierzchnia_corr"].hist()

In [ ]:
df["cena/m"].hist()

In [ ]:
print(df.shape)
print(dummies.shape)

In [ ]:
df.isna().sum()

In [ ]:
#final_plots=final[['powierzchnia_corr','powierzchniaDzialki_corr','rokBudowy_corr','cena/m','locationX','locationY']].copy()
#final_plots=final_plots[final_plots['powierzchnia_corr']<1500]
#final_plots=final_plots[final_plots['powierzchniaDzialki_corr']<5000]
#final_no_dummies=final_no_dummies[final_no_dummies['powierzchnia_corr']<1500]
#final_no_dummies=final_no_dummies[final_no_dummies['powierzchniaDzialki_corr']<5000]

In [ ]:
df.locationX=df.locationX.fillna(10)
df.locationY=df.locationY.fillna(40)

In [ ]:
values=[]
for i in [2,3,4,5,6,7,8,9,10,11,12,13,14]:
  kmeans = KMeans(n_clusters=i, random_state=0).fit(df.values)
  values.append(kmeans.score(df.values))

In [ ]:
print(np.abs(values))
print(values)

In [ ]:
plt.plot(np.abs(values))

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(df.values)


In [ ]:
final_plots['kmean']=kmeans.predict(df.values)

In [ ]:
final_plots.head(10)

In [ ]:
final_plots['powierzchnia_cat']=(final_plots['powierzchnia_corr']/50).astype(int)

In [ ]:
final_plots['powierzchnia_corr'].hist()

In [ ]:
sns.catplot(y="materialBudynku", x="cena/m", hue="kmean", kind="boxen", data=final_plots,height=10)

In [ ]:
sns.catplot(y="rodzajZabudowy", x="cena/m", hue="kmean", kind="boxen", data=final_plots,height=10)

In [ ]:
sns.catplot(x="powierzchnia_cat", y="cena/m", hue="kmean", kind="boxen", data=final_plots,height=10)

In [ ]:
sns.relplot(y="locationY", x="locationX", hue="kmean", data=final_plots,size='cena/m',height=10, sizes=(1, 1000),palette="muted")

In [ ]:
sns.relplot(y="locationY", x="locationX", hue="kmean", data=final_plots[(final_plots["locationX"]>20) & (final_plots["locationX"]<21.5)&
                                                                             (final_plots["locationY"]>52) & (final_plots["locationY"]<52.5)],size='cena/m',height=10, sizes=(1, 1000),palette="muted")